In [16]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Conv1D, AveragePooling2D, Conv2D, RNN, LSTM
from keras.layers import Dense, Dropout, Activation, Flatten
from keras import losses
from gensim.models import KeyedVectors
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


NUM_ONCE_TRAIN = 1000
PADDING_MAXLEN = 500

trained_matrix = KeyedVectors.load_word2vec_format('data_noun_fasttext.vec', encoding='utf-8')
# print(trained_matrix.most_similar('장학', topn=30))

input_file = 'data_noun.txt'
belong_file = 'data_belong_label.txt'
property_file = 'data_property_label.txt'


'''OUTPUT DATA'''
fb = open(belong_file, 'r')
fp = open(property_file, 'r')

lines = fb.readlines()
length_data = len(lines)
length_label = 0

output_train = []
for line in lines:
    lists = [int(x) for x in line.split(" ")]
    lists += [int(x) for x in fp.readline().split(" ")]
    length_label = len(lists)
    output_train.append(lists)

# print(output_train)
output_train = np.array(output_train, dtype=np.int8)
output_train = np.reshape(output_train, (length_data, length_label))
print(output_train.shape)
fb.close()
fp.close()


'''INPUT DATA'''
f = open('data_noun.txt', 'r')
lines = f.readlines()
f.close()
input_train = []
print(len(lines))
maxlen = 0
for line in lines:
    # print(line[0:5])
    length = 0
    word_vector = []
    for word in line.split():
        try:
            if len(word) == 1:    # 한글자 단어 제외
                continue
            length += 1
            word_vector.append(trained_matrix[word])
        except:
            pass
    maxlen = max(maxlen, length)
    input_train.append(word_vector)

print(maxlen)


(13029, 43)
13029
439


In [29]:


'''MODEL'''
model = Sequential()
model.add(LSTM(120, input_shape=(None, 100)))
model.add(Dense(43, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


# input_train = input_train.reshape(input_train.shape[0], 1, 500, 100)
'''SPLIT DATA'''
x_train, x_test, y_train, y_test = train_test_split(input_train, output_train, test_size=0.2, random_state=42)
x_len = len(x_train)
print(x_len)

i = 0
while i < x_len:
    x = x_train[i:i+NUM_ONCE_TRAIN]
    y = y_train[i:i+NUM_ONCE_TRAIN]
    x = pad_sequences(x, maxlen=500)
#     x = x.reshape(x.shape[0],1,500,100)
    model.fit(x, y, batch_size=16, epochs=3, verbose=1)
    print(i)
    i += NUM_ONCE_TRAIN



10423
Epoch 1/3
1000/1000 [==============================] - 29s 29ms/step - loss: 0.4936 - acc: 0.7992
Epoch 2/3
1000/1000 [==============================] - 28s 28ms/step - loss: 0.4263 - acc: 0.8243
Epoch 3/3
1000/1000 [==============================] - 28s 28ms/step - loss: 0.4248 - acc: 0.8243
0
Epoch 1/3
1000/1000 [==============================] - 26s 26ms/step - loss: 0.4198 - acc: 0.8271
Epoch 2/3
1000/1000 [==============================] - 28s 28ms/step - loss: 0.4191 - acc: 0.8271
Epoch 3/3
1000/1000 [==============================] - 34s 34ms/step - loss: 0.4197 - acc: 0.8271
1000
Epoch 1/3
1000/1000 [==============================] - 30s 30ms/step - loss: 0.4257 - acc: 0.8236
Epoch 2/3
1000/1000 [==============================] - 35s 35ms/step - loss: 0.4250 - acc: 0.8236
Epoch 3/3
1000/1000 [==============================] - 32s 32ms/step - loss: 0.4244 - acc: 0.8236
2000
Epoch 1/3
1000/1000 [==============================] - 26s 26ms/step - loss: 0.4268 - acc: 0.8219
Ep

In [57]:
model.save('my_model.h5')

In [56]:

# input_train = pad_sequences(input_train, maxlen=500)
# print(input_train.shape)

x_test = pad_sequences(x_test, maxlen=500)
# x_test = x.test.reshape(x_test.shape[0], 1, 500, 100)
score = model.evaluate(x_test, y_test)
print(score)

MemoryError: 

In [ ]:
model.predict()

In [52]:
test = ["전자", "전기", "장학금"]
test_vector = [[]]
for t in test:
    try:
        if len(t) == 1:    # 한글자 단어 제외
            continue
        test_vector[0].append(trained_matrix[t])
    except:
        pass

In [53]:
# print(np.array(x_test).shape)
print(np.array(test_vector).shape)
test_vector_input = pad_sequences(test_vector, maxlen=500)
print(np.array(test_vector_input).shape)
test_vector_input = test_vector_input.reshape(1, 500, 100)
print(np.array(test_vector_input).shape)

(1, 3, 100)
(1, 500, 100)
(1, 500, 100)


In [54]:
a= model.predict(test_vector_input)

In [55]:
np.set_printoptions(threshold=43, precision=2, suppress=True)
print(a)

[[0.16 0.21 0.03 0.03 0.05 0.05 0.22 0.21 0.16 0.17 0.31 0.   0.3  0.17
  0.19 0.22 0.18 0.32 0.17 0.25 0.05 0.19 0.19 0.18 0.28 0.28 0.29 0.3
  0.07 0.07 0.25 0.03 0.01 0.03 0.22 0.22 0.01 0.29 0.02 0.15 0.17 0.71
  0.  ]]
